In [1]:
# Standard library.
import re
from twitterfunc import tweet_clean
import datetime
import sqlite3
from collections import OrderedDict

# External library.
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk import FreqDist
from wordcloud import WordCloud
import json

In [2]:
def dict_value_sort_return_top(frquency_dict, maxreturn):
    """Sort the dictionary according to values and return a list of top n elements"""
    dictionary_sorted = OrderedDict(sorted(frquency_dict.items(), key=lambda t: t[1], reverse=True))
    # Store top values in an array
    # Change maxCount value to extract top n elements
    count = 0
    top_elements = []
    for k, v in dictionary_sorted.items():
        # Key and value pairs are stored in the form of a tuple in the topWords array
        # Another dictionary is not created here in order to preserve the sorted order
        top_elements.append((k, v))
        count += 1
        if count >= maxreturn:
            break
    return top_elements


In [14]:
# Establish connection to Sqlite3 database.
conn = sqlite3.connect('./bioinfotweet.db')
c = conn.cursor()

# Extract tweets' text from the database followed by filtering and tokenizing.
filteredText = []
rowCount = 0
totalHash = []
totalUsers = []
totalTweetID = []
prog_lang = []
total_lang = {}

# Calculate date.
current_utc = str(datetime.datetime.now(datetime.timezone.utc))
year = current_utc[0:4]
month = current_utc[5:7]

if month == 1: # This is to check for the year change (January month)
    month = 12
    year = int(year) - 1
else:
    month = int(month) - 1

name = "{0}-{1}".format(year, str(month).zfill(2)) # name here means yyyy-mm

# NOTE: update here
# Open txt file with the list of programming languages
file_prog = json.load(json_obj)
for lang, link in file_prog.items():
    prog_lang.append(lang.rstrip())
    tempdict = {lang.rstrip(): 0}
    total_lang.update(tempdict)
    
for ha in totalHash:
    for pro in prog_lang:
        if "#"+pro.lower() == ha:
            total_lang[pro] += 1


totalWords = len(filteredText)
freq = FreqDist(filteredText)
uniqueWords = len(freq)
del filteredText

stopHash = ['#twitter', '#tweeted']  # Hastags of no interest
totalHash[:] = [h for h in totalHash if h not in stopHash]  # Get ride of any cell with stop hashtags
hashFreq = FreqDist(totalHash)
usersFreq = FreqDist(totalUsers)
lang_freq = FreqDist(total_lang)

# # Generate a word cloud image.
# wordcloud = WordCloud(font_path='Actor-Regular.ttf', width=1500, height=500,
#                       max_words=500, stopwords=None, background_color='whitesmoke',
#                       max_font_size=None, font_step=1, mode='RGB',
#                       collocations=True, colormap=None, normalize_plurals=True).generate_from_frequencies(freq)
# imagePath = "/home/bioinformaticsbot/bioinfobot.github.io/images/" + name + '.png'  # Put the actual path of the word cloud image produced in the previous step
# wordcloud.to_file(imagePath)
# imageUrl = "https://bioinfobot.github.io/images/" + name + '.png'



# # Sort and store top n elements in an array
topWords = dict_value_sort_return_top(freq, 20)
del freq  # Delete freq variable to free memory space
hashFreqSorted = dict_value_sort_return_top(hashFreq, 20)
del hashFreq
usersFreqSorted = dict_value_sort_return_top(usersFreq, 20)
del usersFreq
lang_freq_sorted = dict_value_sort_return_top(lang_freq, 20)
del lang_freq
lang_nonzero = []
for l in range(len(lang_freq_sorted)):
    if lang_freq_sorted[l][1] > 0:
        lang_nonzero.append(lang_freq_sorted[l])
        print(total_lang)
print(lang_freq_sorted)
# mainJsonDump = {'PopularLanguages': lang_nonzero}
# # Write a json file
# # jsonPath = '/home/bioinformaticsbot/bioinfobot.github.io/data/' + name + '.json'
# jsonPath = "/" +name + '.json'
# with open(jsonPath, 'w') as wcd:
#     json.dump(mainJsonDump, wcd)

[('A.NET (A#/A sharp)', 0), ('A-0 System', 0), ('A+ (A plus)', 0), ('ABAP', 0), ('ABC', 0), ('ABC ALGOL', 0), ('ACC', 0), ('Accent (Rational Synergy)', 0), ('Ace DASL (Distributed Application Specification Language)', 0), ('Action!', 0), ('ActionScript', 0), ('Actor', 0), ('Ada', 0), ('Adenine (Haystack)', 0), ('AdvPL', 0), ('Agda', 0), ('Agilent VEE (Keysight VEE)', 0), ('Agora', 0), ('AIMMS', 0), ('Aldor', 0)]


In [10]:
total_lang["python"]

KeyError: 'python'